# Scraping data from subreddit <a href='https://www.reddit.com/r/AskDocs/'>r/AskDocs</a> 🩺🥼

In [35]:
import praw
import pandas as pd
from datasets import Dataset
import yake
import math

In [36]:
reddit=praw.Reddit(user_agent=True,
                   client_secret='zS6cuhXCP0b1HCU0aSXl236vu9o9SA',client_id='OzzaZ-aY_0oo4TnMg0fnWA',username='EastLeft',password='randomaccnt278')

In [37]:
def genuineQues(title):
    title=title.lower()
    if('help' in title or 'medical' in title or 'symptoms' in title):
        return 1
    return -1
def genuineComment(reply,role,author):
    
    if(reply=='[deleted]' or reply =='[removed]' or (author=='AutoModerator')):
        return -1
    return 1
def getkeyword(text):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size =1
    deduplication_threshold = 0.9
    numOfKeywords = 1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    return keywords[0]

In [38]:
%%time
subreddit = reddit.subreddit('askdocs')
# Define lists to store data
data = []

# Scraping posts & Comments
for post in subreddit.top(limit= 10000): 
    if(genuineQues(post.title)==1):
        data.append({
            'Type': 'Post',
            'Post_id': post.id,
            'Title': post.title,
            'Author': post.author.name if post.author else 'Unknown',
            'Role':post.author_flair_text,
            'Timestamp': post.created_utc,
            'Text': post.selftext,
            'Score': post.score,
            'Total_comments': post.num_comments,
            'Post_URL': post.url
        })

    # Check if the post has comments
        if post.num_comments > 0:
            c=reddit.submission(url=post.url)
            c.comment_limit=3
            c.comments.replace_more(limit= 0)
            for comment in c.comments:
                data.append({
                'Type': 'Comment',
                'Post_id': post.id,
                'Title': post.title,
                'Author': comment.author.name if comment.author else 'Unknown',
                'Role':comment.author_flair_text,
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Total_comments': 0,
                'Post_URL': None
                })



# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)
df.head(10)

CPU times: user 1.27 s, sys: 177 ms, total: 1.45 s
Wall time: 1min 30s


,Type,Post_id,Title,Author,Role,Timestamp,Text,Score,Total_comments,Post_URL
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,1594030661.0,"Dear users,\n\nRecently there have been nursin...",2773,1,https://www.reddit.com/r/AskDocs/comments/hm52...
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,1681872922.0,I’m the person who posted yesterday because my...,2755,23,https://www.reddit.com/r/AskDocs/comments/12re...
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,None,2023-04-19 02:55:23,Thank you for your submission. **Please note t...,1,0,None
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,2023-04-19 03:34:33,Thank you for the update!! Best wishes to both...,604,0,None
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,None,2023-04-19 02:56:41,[deleted],678,0,None
5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,None,1596052554.0,Original Post:\n\n [https://www.reddit.com/r/A...,2494,11,https://www.reddit.com/r/AskDocs/comments/i07g...
6,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,Eastern_Concentrate8,Layperson/not verified as healthcare professio...,2020-07-29 20:06:30,So glad you found an answer and are on the way...,248,0,None
7,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,jcarberry,Physician | Moderator,2020-07-29 21:26:42,"Thank you for updating, OP! Best of luck in yo...",1,0,None
8,Comment,i07g60,*SOLVED* After two years of mystery diagnosis ...,graye1999,This user has not yet been verified.,2020-07-29 20:15:35,That’s fascinating. I would have had a similar...,108,0,None
9,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,1575073307.0,Please take the time to read this. I am extre...,2134,236,https://www.reddit.com/r/AskDocs/comments/e3nr...


In [39]:
df.to_csv("askdocs.csv",index=False)

# Data preprocessing

In [56]:
import praw
import pandas as pd
from datasets import Dataset

In [57]:
df=pd.read_csv('askdocs.csv')
df

,Type,Post_id,Title,Author,Role,Timestamp,Text,Score,Total_comments,Post_URL
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,1594030661.0,"Dear users,\n\nRecently there have been nursin...",2773,1,https://www.reddit.com/r/AskDocs/comments/hm52...
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,1681872922.0,I’m the person who posted yesterday because my...,2755,23,https://www.reddit.com/r/AskDocs/comments/12re...
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,2023-04-19 02:55:23,Thank you for your submission. **Please note t...,1,0,NaN
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,2023-04-19 03:34:33,Thank you for the update!! Best wishes to both...,604,0,NaN
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,2023-04-19 02:56:41,[deleted],678,0,NaN
...,...,...,...,...,...,...,...,...,...,...
489,Comment,ychr4t,Bad burn wound- no health insurance. Please help!,kittencalledmeow,Physician,2022-10-24 18:55:50,This appears to be second degree bc of the bli...,687,0,NaN
490,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Layperson/not verified as healthcare professional,1714211022.0,"27M 190lbs 183cm. For sometime now, and I me...",373,130,https://www.reddit.com/r/AskDocs/comments/1cea...
491,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",AutoModerator,NaN,2024-04-27 09:43:43,Thank you for your submission. **Please note t...,1,0,NaN
492,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",Snownyann,Physician - Pathology,2024-04-27 11:14:50,"Hello, you have severe itching at the anus at ...",1152,0,NaN


In [58]:
df = df.drop('Timestamp',axis=1)
df = df.drop('Total_comments', axis=1)
df = df.drop('Post_URL', axis=1)
df

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,I’m the person who posted yesterday because my...,2755
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,604
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[deleted],678
...,...,...,...,...,...,...,...
489,Comment,ychr4t,Bad burn wound- no health insurance. Please help!,kittencalledmeow,Physician,This appears to be second degree bc of the bli...,687
490,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Layperson/not verified as healthcare professional,"27M 190lbs 183cm. For sometime now, and I me...",373
491,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",AutoModerator,NaN,Thank you for your submission. **Please note t...,1
492,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",Snownyann,Physician - Pathology,"Hello, you have severe itching at the anus at ...",1152


In [59]:
df.drop(df[df.Author =='AutoModerator'].index)
df.drop(df[df.Text =='[deleted]'].index)
df.drop(df[df.Text =='[removed]'].index)

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users,\n\nRecently there have been nursin...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Layperson/not verified as healthcare professional,I’m the person who posted yesterday because my...,2755
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank you for your submission. **Please note t...,1
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank you for the update!! Best wishes to both...,604
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[deleted],678
...,...,...,...,...,...,...,...
489,Comment,ychr4t,Bad burn wound- no health insurance. Please help!,kittencalledmeow,Physician,This appears to be second degree bc of the bli...,687
490,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Layperson/not verified as healthcare professional,"27M 190lbs 183cm. For sometime now, and I me...",373
491,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",AutoModerator,NaN,Thank you for your submission. **Please note t...,1
492,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",Snownyann,Physician - Pathology,"Hello, you have severe itching at the anus at ...",1152


In [60]:
df['Role'] = df['Role'].where(df['Role'] != 'Layperson/not verified as healthcare professional', 'Patient')
df['Role'] = df['Role'].where(df['Role'] != 'This user has not yet been verified.', 'Patient')

In [61]:
import spacy
from summa import keywords
# Load spaCy English model
key=[]
nlp = spacy.load("en_core_web_sm")
for i in range(len(df)):
    doc=nlp(df.iloc[i]['Text'])
    filtered_words = [token.text for token in doc if not token.is_stop]
    clean_text = ' '.join(filtered_words)
    df.loc[i,"Text"]=clean_text
    TR_keywords = keywords.keywords(clean_text, scores=True)
    TR_keywords=TR_keywords[0:2]
    for tup in TR_keywords:
        if(tup[0] not in key):
            key.append(tup[0])



In [62]:
df

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank submission . * * note response constitut...,1
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank update ! ! Best wishes - healing comfort !,604
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[ deleted ],678
...,...,...,...,...,...,...,...
489,Comment,ychr4t,Bad burn wound- no health insurance. Please help!,kittencalledmeow,Physician,appears second degree bc blistering . Burns te...,687
490,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Patient,"27 M 190lbs 183 cm . , mean year , anus itc...",373
491,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",AutoModerator,NaN,Thank submission . * * note response constitut...,1
492,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",Snownyann,Physician - Pathology,"Hello , severe itching anus night ? thing came...",1152


In [63]:
def word_appears_in_files(word_list, file_paths):
    imp=[]
        # Iterate through each file path
    for file_path in file_paths:
            # Open the file in read mode
        with open(file_path, 'r') as file:
                # Read the contents of the file
            file_contents = file.read()
                
                # Check if any word from the list appears in the file
            for word in word_list:
                if word in file_contents:
                    imp.append(word)  # Return True if word is found in file, along with file path
        # If none of the words are found in any file
    return imp

# Example usage
file_paths = [ "disease.en.txt"]  # Replace with the paths to your files

# Check if any word from the list appears in one of the files
l = word_appears_in_files(key, file_paths)
print(l)


['post', 'best', 'oral', 'esophageal', 'syndrome', 'test', 'dog', 'wear', 'brain', 'run', 'caused', 'right', 'like', 'know', 'road', 'aneurysm', 'time', 'gyne', 'hope', 'pain', 'painful', 'rape', 'blood', 'food', 'gastroparesis', 'note', 'vaccine', 'symptoms', 'bet', 'edit', 'diarrhea', 'pediatric', 'bleeding', 'fibroid', 'hour', 'inflammation', 'case', 'non', 'new', 'disorder', 'er', 'x', 'bit', 'think', 'eyes', 'eye', 'attention', 'posts', 'age', 'bladder', 'difficult', 'vomiting', 'rectum', 'ct', 'link', 'op', 'brain tumor', 'family', 'survival', 'diabetes', 'life', 'heart', 'chest', 'morphine', 'mom', 'seizures', 'period', 'functional', 'good', 'psychosis', 'dr', 'sexual', 'disease', 'puberty', 'hard', 'weight', 'baby', 'hands', 'mask', 'city', 'issues', 'chronic', 'drug', 'activate', 'dementia', 'contact', 'came', 'rheumatoid', 'pt', 'home', 'stoma', 'bowel', 'bf', 'seizure', 'attack', 'attacks', 'obstruction', 'cancer', 'infection', 'liver', 'arm', 'ventriculomegaly', 'ulcerative

In [64]:
i=0
while(i<len(df)-1):
    if(df['Type'][i]=='Post'):
        j=i+1
        while(j<len(df) and df['Type'][j]=='Comment'):
            reply=df['Text'][j]
            role=df['Role'][j]
            author=df['Author'][j]
            if((genuineComment(reply,role,author))==-1):
                df.drop(j)
            j+=1
    i+=1
df

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755
2,Comment,12rebr3,Update on my husband with drooping mouth/other...,AutoModerator,NaN,Thank submission . * * note response constitut...,1
3,Comment,12rebr3,Update on my husband with drooping mouth/other...,NoBasil3262,Registered Nurse,Thank update ! ! Best wishes - healing comfort !,604
4,Comment,12rebr3,Update on my husband with drooping mouth/other...,Unknown,NaN,[ deleted ],678
...,...,...,...,...,...,...,...
489,Comment,ychr4t,Bad burn wound- no health insurance. Please help!,kittencalledmeow,Physician,appears second degree bc blistering . Burns te...,687
490,Post,1ceap60,"Itchy anus, causing me to insert my finger in ...",throwitaway1231082,Patient,"27 M 190lbs 183 cm . , mean year , anus itc...",373
491,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",AutoModerator,NaN,Thank submission . * * note response constitut...,1
492,Comment,1ceap60,"Itchy anus, causing me to insert my finger in ...",Snownyann,Physician - Pathology,"Hello , severe itching anus night ? thing came...",1152


In [65]:
i=0
df = df.drop(df[df.Type == 'Comment'].index)
i+=1
df

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755
5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post : \n\n [ https://www.reddit.com...,2494
9,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,time read . extremely sick sure going surviv...,2134
13,Post,ru1vrj,Am I at imminent risk of death? I’m scared. Pl...,Throwaway579z,Patient,"18F , BMI 12.2 , located \n\n medications \n\n...",1802
...,...,...,...,...,...,...,...
476,Post,mxe8oh,"I have a fingernail stuck in my nose, please help",dumbnosegirl,Layperson/not verified as healthcare professio...,"15F , 5'2 "" , 120 lbs , white , medications ...",395
480,Post,d2bluk,[26F] I have no acess to healthcare and need h...,idontwannadiehelp,Layperson/not verified as healthcare professio...,"Warning : English language , bad grammar ahead...",387
484,Post,en76pf,Being told by surgeon and other medical profes...,hemorrhoidbegone22,Layperson/not verified as healthcare professio...,Hello . \n\n stats : \n\n 32 M \n\n 6'0 \n\n 2...,382
487,Post,ychr4t,Bad burn wound- no health insurance. Please help!,dreiscray,Patient,Hello thanks taking time help . ( 28F 5’0 125 ...,385


In [66]:
df.to_csv('final.csv',index=False)
df

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755
5,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post : \n\n [ https://www.reddit.com...,2494
9,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,time read . extremely sick sure going surviv...,2134
13,Post,ru1vrj,Am I at imminent risk of death? I’m scared. Pl...,Throwaway579z,Patient,"18F , BMI 12.2 , located \n\n medications \n\n...",1802
...,...,...,...,...,...,...,...
476,Post,mxe8oh,"I have a fingernail stuck in my nose, please help",dumbnosegirl,Layperson/not verified as healthcare professio...,"15F , 5'2 "" , 120 lbs , white , medications ...",395
480,Post,d2bluk,[26F] I have no acess to healthcare and need h...,idontwannadiehelp,Layperson/not verified as healthcare professio...,"Warning : English language , bad grammar ahead...",387
484,Post,en76pf,Being told by surgeon and other medical profes...,hemorrhoidbegone22,Layperson/not verified as healthcare professio...,Hello . \n\n stats : \n\n 32 M \n\n 6'0 \n\n 2...,382
487,Post,ychr4t,Bad burn wound- no health insurance. Please help!,dreiscray,Patient,Hello thanks taking time help . ( 28F 5’0 125 ...,385


### Hugging face: Using Zero Shot to label the data

In [7]:
from transformers import pipeline
import pandas as pd
newcsv=pd.read_csv('final.csv')
newcsv

,Type,Post_id,Title,Author,Role,Text,Score
0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773
1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755
2,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post : \n\n [ https://www.reddit.com...,2494
3,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,time read . extremely sick sure going surviv...,2134
4,Post,ru1vrj,Am I at imminent risk of death? I’m scared. Pl...,Throwaway579z,Patient,"18F , BMI 12.2 , located \n\n medications \n\n...",1802
...,...,...,...,...,...,...,...
136,Post,mxe8oh,"I have a fingernail stuck in my nose, please help",dumbnosegirl,Layperson/not verified as healthcare professio...,"15F , 5'2 "" , 120 lbs , white , medications ...",395
137,Post,d2bluk,[26F] I have no acess to healthcare and need h...,idontwannadiehelp,Layperson/not verified as healthcare professio...,"Warning : English language , bad grammar ahead...",387
138,Post,en76pf,Being told by surgeon and other medical profes...,hemorrhoidbegone22,Layperson/not verified as healthcare professio...,Hello . \n\n stats : \n\n 32 M \n\n 6'0 \n\n 2...,382
139,Post,ychr4t,Bad burn wound- no health insurance. Please help!,dreiscray,Patient,Hello thanks taking time help . ( 28F 5’0 125 ...,385


In [69]:

zs_class = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")
i=0
while(i<len(df)-1):
    text=""
    text=newcsv['Title'][i]+' '+newcsv['Text'][i]
    lc  = zs_class(text, l)
    print(lc['labels'][0])
    newcsv['Label']=lc['labels'][0]
    i+=1

associated
symptoms
associated
sick
associated
positive
edit
think
ventriculomegaly
associated
positive
associated
edit
painful
sexual
associated
associated
sick
associated
possible
possible
associated
edit
sick
painful
sick
bleeding
infection
associated
associated
painful
gastroparesis
eyes
associated
associated
possible
hearing loss
associated
possible
sick
diagnosed
possible
ulcerative
diagnosed
hard
associated
associated
heart
associated
chronic
seizures
possible
painful
associated
time
particular
associated
associated
associated
symptoms
edit
associated
rough
associated
associated
edit
chronic
associated
bleeding
hard
link
obesity
associated
sick
hearing
associated
possible
associated
sick
diagnosed
diagnosed
f pregnancy
mom
associated
case
associated
associated
bleeding
associated
associated
painful
sick
family
wound
associated
associated
hearing
associated
possible
associated
associated
family
associated
painful
possible
sick
diagnosed
painful
painful
lot
family
associated
sick


In [9]:
labelY=[]
with open("labelsoutput.txt") as file:
  for item in file:

    labelY.append(item[:len(item)-1])
labelY

['associated',
 'symptoms',
 'associated',
 'sick',
 'associated',
 'positive',
 'edit',
 'think',
 'ventriculomegaly',
 'associated',
 'positive',
 'associated',
 'edit',
 'painful',
 'sexual',
 'associated',
 'associated',
 'sick',
 'associated',
 'possible',
 'possible',
 'associated',
 'edit',
 'sick',
 'painful',
 'sick',
 'bleeding',
 'infection',
 'associated',
 'associated',
 'painful',
 'gastroparesis',
 'eyes',
 'associated',
 'associated',
 'possible',
 'hearing loss',
 'associated',
 'possible',
 'sick',
 'diagnosed',
 'possible',
 'ulcerative',
 'diagnosed',
 'hard',
 'associated',
 'associated',
 'heart',
 'associated',
 'chronic',
 'seizures',
 'possible',
 'painful',
 'associated',
 'time',
 'particular',
 'associated',
 'associated',
 'associated',
 'symptoms',
 'edit',
 'associated',
 'rough',
 'associated',
 'associated',
 'edit',
 'chronic',
 'associated',
 'bleeding',
 'hard',
 'link',
 'obesity',
 'associated',
 'sick',
 'hearing',
 'associated',
 'possible',
 'as

In [12]:
newcsv['Label'] = labelY
newcsv.to_csv("finalfinal.csv")

In [14]:
dataf=pd.read_csv("finalfinal.csv")
dataf

,Unnamed: 0,Type,Post_id,Title,Author,Role,Text,Score,Label
0,0,Post,hm525p,Announcement: only medical students will be ve...,murpahurp,Physician | Moderator | Top Contributor,"Dear users , \n\n Recently nursing schools swi...",2773,associated
1,1,Post,12rebr3,Update on my husband with drooping mouth/other...,imeanyoucouldsure,Patient,"person posted yesterday husband ( 42 male , 32...",2755,symptoms
2,2,Post,i07g60,*SOLVED* After two years of mystery diagnosis ...,Unknown,NaN,Original Post : \n\n [ https://www.reddit.com...,2494,associated
3,3,Post,e3nrbd,Need help - Harvard gave up - homeless,gooddiagnostician,Layperson/not verified as healthcare professio...,time read . extremely sick sure going surviv...,2134,sick
4,4,Post,ru1vrj,Am I at imminent risk of death? I’m scared. Pl...,Throwaway579z,Patient,"18F , BMI 12.2 , located \n\n medications \n\n...",1802,associated
...,...,...,...,...,...,...,...,...,...
136,136,Post,mxe8oh,"I have a fingernail stuck in my nose, please help",dumbnosegirl,Layperson/not verified as healthcare professio...,"15F , 5'2 "" , 120 lbs , white , medications ...",395,anxiety
137,137,Post,d2bluk,[26F] I have no acess to healthcare and need h...,idontwannadiehelp,Layperson/not verified as healthcare professio...,"Warning : English language , bad grammar ahead...",387,associated
138,138,Post,en76pf,Being told by surgeon and other medical profes...,hemorrhoidbegone22,Layperson/not verified as healthcare professio...,Hello . \n\n stats : \n\n 32 M \n\n 6'0 \n\n 2...,382,painful
139,139,Post,ychr4t,Bad burn wound- no health insurance. Please help!,dreiscray,Patient,Hello thanks taking time help . ( 28F 5’0 125 ...,385,arm


One challenge is that the ideal answer could be quite inside a conversation of a thread.

# Fine Tuning